We will use the daily spreadsheet from EU CDC containing new cases and deaths per country per day.

In [1]:
!wget -N https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx

--2020-06-02 12:34:54--  https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx
Resolving www.ecdc.europa.eu (www.ecdc.europa.eu)... 2600:9000:21c7:7200:1d:d83a:40c0:93a1, 2600:9000:21c7:4600:1d:d83a:40c0:93a1, 2600:9000:21c7:200:1d:d83a:40c0:93a1, ...
Connecting to www.ecdc.europa.eu (www.ecdc.europa.eu)|2600:9000:21c7:7200:1d:d83a:40c0:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 968511 (946K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘COVID-19-geographic-disbtribution-worldwide.xlsx’

COVID-19-geographic 100%[===================>] 945.81K  --.-KB/s    in 0.1s    

2020-06-02 12:34:54 (7.88 MB/s) - ‘COVID-19-geographic-disbtribution-worldwide.xlsx’ saved [968511/968511]



Get Pandas and NumPy for feature engineering and calculations and get plots inline.

In [2]:
import pandas as pd
import numpy  as np

%matplotlib inline

We read our dataframe directly from the downloaded Excel file and have a look at the first 10 lines for format. Data for Namibia caused missing values because the `geoId` is __NA__, so we disable interpretation of missing values.

In [3]:
df = pd.read_excel('COVID-19-geographic-disbtribution-worldwide.xlsx', keep_default_na=False, na_values='')
df.head(10)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,continentExp
0,2020-06-01,1,6,2020,680,8,Afghanistan,AF,AFG,37172386.0,Asia
1,2020-05-31,31,5,2020,866,3,Afghanistan,AF,AFG,37172386.0,Asia
2,2020-05-30,30,5,2020,623,11,Afghanistan,AF,AFG,37172386.0,Asia
3,2020-05-29,29,5,2020,580,8,Afghanistan,AF,AFG,37172386.0,Asia
4,2020-05-28,28,5,2020,625,7,Afghanistan,AF,AFG,37172386.0,Asia
5,2020-05-27,27,5,2020,658,1,Afghanistan,AF,AFG,37172386.0,Asia
6,2020-05-26,26,5,2020,591,1,Afghanistan,AF,AFG,37172386.0,Asia
7,2020-05-25,25,5,2020,584,2,Afghanistan,AF,AFG,37172386.0,Asia
8,2020-05-24,24,5,2020,782,11,Afghanistan,AF,AFG,37172386.0,Asia
9,2020-05-23,23,5,2020,540,12,Afghanistan,AF,AFG,37172386.0,Asia


Last check of our source dataframe.

In [4]:
df.count()

dateRep                    20711
day                        20711
month                      20711
year                       20711
cases                      20711
deaths                     20711
countriesAndTerritories    20711
geoId                      20711
countryterritoryCode       20487
popData2018                20415
continentExp               20711
dtype: int64

We pivot to a country by column format.

In [5]:
df_geo = df.pivot(index='dateRep', columns='geoId', values=['cases', 'deaths'])
df_geo

cases                                                         ...  \
geoId         AD     AE     AF   AG   AI    AL     AM   AO     AR    AT  ...   
dateRep                                                                  ...   
2019-12-31   NaN    0.0    0.0  NaN  NaN   NaN    0.0  NaN    NaN   0.0  ...   
2020-01-01   NaN    0.0    0.0  NaN  NaN   NaN    0.0  NaN    NaN   0.0  ...   
2020-01-02   NaN    0.0    0.0  NaN  NaN   NaN    0.0  NaN    NaN   0.0  ...   
2020-01-03   NaN    0.0    0.0  NaN  NaN   NaN    0.0  NaN    NaN   0.0  ...   
2020-01-04   NaN    0.0    0.0  NaN  NaN   NaN    0.0  NaN    NaN   0.0  ...   
...          ...    ...    ...  ...  ...   ...    ...  ...    ...   ...  ...   
2020-05-28   0.0  883.0  625.0  0.0  0.0  21.0  372.0  0.0  705.0  18.0  ...   
2020-05-29   0.0  563.0  580.0  0.0  0.0  26.0  442.0  2.0  769.0  28.0  ...   
2020-05-30   1.0  638.0  623.0  0.0  0.0  23.0  460.0  4.0  717.0  51.0  ...   
2020-05-31   0.0  726.0  866.0  0.0  0.0  23.0  251.0  7.0  795.0  44.0  ...   
2020-06-01   0.0  661.0  680.0  0.0  0.0  15.0  355.0  2.0  637.0   4.0  ...   

           deaths                                                 
geoId          VC   VE   VG   VI   VN   XK    YE    ZA   ZM   ZW  
dateRep                                                           
2019-12-31    NaN  NaN  NaN  NaN  0.0  NaN   NaN   NaN  NaN  NaN  
2020-01-01    NaN  NaN  NaN  NaN  0.0  NaN   NaN   NaN  NaN  NaN  
2020-01-02    NaN  NaN  NaN  NaN  0.0  NaN   NaN   NaN  NaN  NaN  
2020-01-03    NaN  NaN  NaN  NaN  0.0  NaN   NaN   NaN  NaN  NaN  
2020-01-04    NaN  NaN  NaN  NaN  0.0  NaN   NaN   NaN  NaN  NaN  
...           ...  ...  ...  ...  ...  ...   ...   ...  ...  ...  
2020-05-28    0.0  0.0  0.0  0.0  0.0  0.0   4.0  28.0  0.0  0.0  
2020-05-29    0.0  0.0  0.0  0.0  0.0  0.0   4.0  25.0  0.0  0.0  
2020-05-30    0.0  3.0  0.0  0.0  0.0  0.0   9.0  34.0  0.0  0.0  
2020-05-31    0.0  0.0  0.0  0.0  0.0  0.0  11.0  32.0  0.0  0.0  
2020-06-01    0.0  0.0  0.0  0.0  0.0  0.0   3.0  40.0  0.0  0.0  

[154 rows x 420 columns]

For predictions later on we need extra rows in our dataframe. One of the ways to do that is reindexing with a larger range, so we use the current range and add six months and check our latest date.

In [6]:
new_index = pd.date_range(df_geo.index.min(), df_geo.index.max() + pd.Timedelta('365 days'))
df_geo = df_geo.reindex(new_index)
df_geo

cases                                          ... deaths          \
geoId         AD   AE   AF  AG  AI  AL   AM  AO  AR   AT  ...     VC  VE  VG   
2019-12-31   NaN  0.0  0.0 NaN NaN NaN  0.0 NaN NaN  0.0  ...    NaN NaN NaN   
2020-01-01   NaN  0.0  0.0 NaN NaN NaN  0.0 NaN NaN  0.0  ...    NaN NaN NaN   
2020-01-02   NaN  0.0  0.0 NaN NaN NaN  0.0 NaN NaN  0.0  ...    NaN NaN NaN   
2020-01-03   NaN  0.0  0.0 NaN NaN NaN  0.0 NaN NaN  0.0  ...    NaN NaN NaN   
2020-01-04   NaN  0.0  0.0 NaN NaN NaN  0.0 NaN NaN  0.0  ...    NaN NaN NaN   
...          ...  ...  ...  ..  ..  ..  ...  ..  ..  ...  ...    ...  ..  ..   
2021-05-28   NaN  NaN  NaN NaN NaN NaN  NaN NaN NaN  NaN  ...    NaN NaN NaN   
2021-05-29   NaN  NaN  NaN NaN NaN NaN  NaN NaN NaN  NaN  ...    NaN NaN NaN   
2021-05-30   NaN  NaN  NaN NaN NaN NaN  NaN NaN NaN  NaN  ...    NaN NaN NaN   
2021-05-31   NaN  NaN  NaN NaN NaN NaN  NaN NaN NaN  NaN  ...    NaN NaN NaN   
2021-06-01   NaN  NaN  NaN NaN NaN NaN  NaN NaN NaN  NaN  ...    NaN NaN NaN   

                                         
geoId       VI   VN  XK  YE  ZA  ZM  ZW  
2019-12-31 NaN  0.0 NaN NaN NaN NaN NaN  
2020-01-01 NaN  0.0 NaN NaN NaN NaN NaN  
2020-01-02 NaN  0.0 NaN NaN NaN NaN NaN  
2020-01-03 NaN  0.0 NaN NaN NaN NaN NaN  
2020-01-04 NaN  0.0 NaN NaN NaN NaN NaN  
...         ..  ...  ..  ..  ..  ..  ..  
2021-05-28 NaN  NaN NaN NaN NaN NaN NaN  
2021-05-29 NaN  NaN NaN NaN NaN NaN NaN  
2021-05-30 NaN  NaN NaN NaN NaN NaN NaN  
2021-05-31 NaN  NaN NaN NaN NaN NaN NaN  
2021-06-01 NaN  NaN NaN NaN NaN NaN NaN  

[519 rows x 420 columns]

Most algorithms take numerical data as inputs for a model, so we add a column representing the date as days since the earliest date in the dataframe.

In [7]:
df_geo['daynum'] = (df_geo.index - df_geo.index.min()).days
df_geo['daynum'].describe()

count    519.000000
mean     259.000000
std      149.966663
min        0.000000
25%      129.500000
50%      259.000000
75%      388.500000
max      518.000000
Name: daynum, dtype: float64

Suppress warnings for multiple plots when analyzing many countries with `showplots = True`.

In [8]:
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)

Running for multiple countries with a selection or simply all countries found in the input. Full documentation of the approach is found in the `Gumbelpivot` notebook.

In [12]:
# Select countries to fit.
countries = np.sort(df['geoId'].unique())
#countries = ['US', 'UK', 'BR', 'CH', 'DE', 'IT', 'ES', 'PT', 'FR', 'SE', 'NO', 'DK', 'BE', 'NL', 'NZ', 'CN', 'JP', 'RU', 'AT']

# Choose whether to output plots per country.
showplots = False

# Create an output dataframe.
df_out = pd.DataFrame({
    'cname':np.nan,
    'iso3':np.nan,
    'ccont':np.nan,
    'popdata':np.nan,
    'rsquared':np.nan,
    'progress':np.nan,
    'final':np.nan,
    'start':np.nan,
    'peak':np.nan,
    'floor':np.nan,
    'beta':np.nan,
    'mu':np.nan,
    'maxcur':np.nan},
    index=countries)

# Choose measure to fit and variables to store predicted and smoothed measures.
measure  = 'cases'
smeasure = 'scases'
pmeasure = 'pcases'

def gumbelval(x, beta, mu):
    """Return the Gumbel CDF for x according to beta and mu"""
    return np.exp(- np.exp(- (x - mu) / beta))

def gumbelinv(x):
    """Inverse Gumbel function"""
    return(- np.log(- np.log(x)))

from scipy.stats import linregress

def fitres(progress):
    """Try to fit a line according to progress, returning correlation of fit"""
    global df_pred, slope, intercept
    
    # Scale the cumulative measure and only keep cases below 1 for fitting
    df_pred['scaled'] = df_pred['cumul'] / numcases * progress
    df_fit = df_pred[df_pred['scaled'] < 1].copy()
    
    # Only try fitting if we have at least 5 measures left.
    if len(df_fit) > 4:
        df_fit['linear'] = gumbelinv(df_fit['scaled'])
        slope, intercept, correlation, pvalue, stderr = linregress(df_fit[['daynum', 'linear']])
        #print('Progress {:13.9f} gives {:13.9f} for {:1.0f} measures'.format(
        #    progress, fit[1][0], len(df_fit)))
        return(1 - correlation)
    else:
        return np.nan

from scipy.optimize import minimize_scalar
    
# Run the fitting approach for all countries.
for country in countries:
    df_geo[(smeasure, country)] = df_geo[measure][country].rolling(7).mean()
    df_pred = pd.DataFrame(
        {'daynum':df_geo['daynum'], measure:df_geo[smeasure][country]})
    
    # Extract country parameters from the original dataset.
    cname   = df[df['geoId'] == country]['countriesAndTerritories'].iloc[0]
    iso3    = df[df['geoId'] == country]['countryterritoryCode'].iloc[0]
    ccont   = df[df['geoId'] == country]['continentExp'].iloc[0]
    popdata = df[df['geoId'] == country]['popData2018'].iloc[0]

    # Current number of cases for scaling.
    numcases = df_pred[measure].sum()
    
    # We will only use measures above one in a million.
    mincases = popdata / 1e6
    df_pred = df_pred[df_pred[measure] > mincases]

    # Only start fitting if we have at least 5 measures.
    if len(df_pred) > 4:
        df_pred['cumul'] = df_pred[measure].cumsum()
        
        # Find the optimal fit.
        optim    = minimize_scalar(fitres, method='bounded', bounds=(0, 1.5))
        progress = optim.x
        rsquared = (1 - optim.fun) ** 2
        bestfit  = fitres(progress)
        
        # Calculate Gumbel beta and mu from our linear fit parameters.
        beta = 1 / slope
        mu = - intercept / slope
        
        # Create predicted measures by calculating the Gumbel CDF and reduce to PDF.
        df_geo[(pmeasure, country)] = np.gradient(gumbelval(df_geo['daynum'], beta, mu) * numcases / progress)
 
        # Determine peak, floor, start and final analytically.
        peak = df_geo[(df_geo[(pmeasure, country)] > df_geo[(pmeasure, country)].shift(-1))].index.min()
        floor = df_geo[(df_geo[(pmeasure, country)] < (popdata / 1e6)) & (
            df_geo[(pmeasure, country)].index > peak)].index.min()
        start = df_geo[(df_geo[(pmeasure, country)] > (popdata / 1e6)) & (
            df_geo[(pmeasure, country)].index < peak)].index.min()
        final = df_geo[pmeasure][country].sum()
        
        # Maximum current infected seems a good measure for outbreak intensity, to be scaled by population.
        maxcur = df_geo[pmeasure][country].rolling(14).sum().max()
        
        # Create an output record and log results.
        df_out.loc[country] = [cname, iso3, ccont, popdata, rsquared, progress, final, start.date(), peak.date(), floor.date(), beta, mu, maxcur]
        print('{}: rsquared {:5.3f} at {:3.0f}% of {:7.0f} start {} peak {} floor {} beta {:5.2f} mu {:3.0f}'.format(
            country, rsquared, progress * 100, final, start.date(), peak.date(), floor.date(), beta, mu))
        
        # Show cumulative and derived results.
        if showplots:
            df_geo[[(measure, country), (smeasure, country), (pmeasure, country)]].cumsum().plot(
                figsize=(16, 9), grid=True)
            df_geo[[(measure, country), (smeasure, country), (pmeasure, country)]].plot(
                figsize=(16, 9), grid=True)
    else:
        df_out.loc[country] = [cname, iso3, ccont, popdata, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

AD: rsquared 0.955 at 114% of     637 start 2020-03-18 peak 2020-03-31 floor 2020-05-15 beta  6.15 mu  91
AE: rsquared 0.994 at  67% of   48670 start 2020-03-22 peak 2020-05-12 floor 2020-09-25 beta 25.67 mu 133
AF: rsquared 0.995 at  18% of   74889 start 2020-04-18 peak 2020-06-25 floor 2020-11-28 beta 39.71 mu 177
AG: rsquared 0.983 at  99% of      24 start 2020-03-23 peak 2020-04-05 floor 2020-05-02 beta  7.93 mu  96
AL: rsquared 0.994 at  91% of    1172 start 2020-03-14 peak 2020-04-16 floor 2020-06-20 beta 22.74 mu 107
AM: rsquared 0.972 at  27% of   30601 start 2020-03-09 peak 2020-06-23 floor 2021-04-03 beta 53.98 mu 175
AR: rsquared 0.969 at  31% of   47657 start 2020-04-01 peak 2020-06-17 floor 2020-11-18 beta 51.01 mu 169
AT: rsquared 0.992 at 104% of   15844 start 2020-03-09 peak 2020-03-28 floor 2020-05-19 beta  9.83 mu  88
AU: rsquared 0.998 at 108% of    6632 start 2020-03-17 peak 2020-03-28 floor 2020-04-23 beta  7.04 mu  88
AW: rsquared 0.972 at 100% of      49 start 20

IQ: rsquared 0.945 at  37% of   14686 start 2020-04-17 peak 2020-06-16 floor 2020-09-21 beta 53.55 mu 168
IR: rsquared 0.982 at  87% of  166258 start 2020-02-24 peak 2020-04-15 floor 2020-08-15 beta 28.55 mu 106
IS: rsquared 0.982 at 100% of    1813 start 2020-03-06 peak 2020-03-24 floor 2020-05-21 beta  8.95 mu  84
IT: rsquared 1.000 at  97% of  237761 start 2020-02-28 peak 2020-04-01 floor 2020-07-03 beta 16.95 mu  92
JE: rsquared 0.975 at 128% of     231 start 2020-03-24 peak 2020-04-05 floor 2020-05-10 beta  5.98 mu  96
JM: rsquared 0.976 at 106% of     528 start 2020-04-07 peak 2020-04-24 floor 2020-05-26 beta 11.94 mu 115
JO: rsquared 0.850 at 123% of     571 start NaT peak 2020-04-09 floor 2020-04-10 beta 28.65 mu 100
JP: rsquared 0.999 at 110% of   15155 start 2020-04-04 peak 2020-04-17 floor 2020-05-11 beta 10.03 mu 108
JPG11668: rsquared 0.990 at  82% of     856 start 2020-01-31 peak 2020-02-18 floor 2020-05-03 beta  7.07 mu  49
KE: rsquared 0.994 at  42% of    3861 start 202

SX: rsquared 0.958 at 114% of      52 start 2020-04-02 peak 2020-04-12 floor 2020-05-10 beta  5.07 mu 103
SZ: rsquared 0.983 at  93% of     295 start 2020-04-22 peak 2020-05-10 floor 2020-06-15 beta 12.93 mu 131
TC: rsquared 0.980 at  96% of      10 start 2020-03-24 peak 2020-04-07 floor 2020-05-09 beta  8.87 mu  98
TD: rsquared 0.994 at  96% of     761 start 2020-05-08 peak 2020-05-16 floor 2020-05-30 beta  9.07 mu 137
TG: rsquared 0.985 at 150% of     275 start 2020-05-13 peak 2020-05-19 floor 2020-05-28 beta  5.83 mu 140
TH: rsquared 0.995 at 111% of    2693 start 2020-03-24 peak 2020-03-31 floor 2020-04-11 beta  7.72 mu  91
TJ: rsquared 0.995 at  50% of    6709 start 2020-05-03 peak 2020-05-27 floor 2020-07-22 beta 13.89 mu 148
TL: rsquared 0.999 at  90% of      26 start 2020-04-17 peak 2020-04-20 floor 2020-04-26 beta  3.54 mu 111
TN: rsquared 0.995 at 113% of     937 start 2020-03-26 peak 2020-04-06 floor 2020-04-25 beta  9.20 mu  97
TR: rsquared 0.998 at  96% of  168170 start 20

Check the output frame assigning the index name.

In [13]:
df_out.index.name = 'iso2'
df_out

,cname,iso3,ccont,popdata,rsquared,progress,final,start,peak,floor,beta,mu,maxcur
iso2,,,,,,,,,,,,,
AD,Andorra,AND,Europe,77006.0,0.955327,1.144845,637.141572,2020-03-18,2020-03-31,2020-05-15,6.149978,91.156126,439.684610
AE,United_Arab_Emirates,ARE,Asia,9630959.0,0.994398,0.666050,48669.864101,2020-03-22,2020-05-12,2020-09-25,25.670654,132.693121,9641.819054
AF,Afghanistan,AFG,Asia,37172386.0,0.994937,0.175031,74888.936321,2020-04-18,2020-06-25,2020-11-28,39.706238,176.920979,9664.442637
AG,Antigua_and_Barbuda,ATG,America,96286.0,0.983465,0.990206,23.660298,2020-03-23,2020-04-05,2020-05-02,7.927100,95.526361,13.604696
AI,Anguilla,NaN,America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XK,Kosovo,XKX,Europe,1845300.0,0.992235,0.935607,1108.524566,2020-03-22,2020-04-18,2020-06-17,16.698912,109.349854,331.986753
YE,Yemen,YEM,Asia,28498687.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZA,South_Africa,ZAF,Africa,57779622.0,0.978311,0.035563,748449.237279,2020-04-05,2020-09-11,NaT,81.831457,254.968956,48963.953206


Write out the values per country, discarding countries with progress below 1%.

In [14]:
df_out[df_out['progress'] > 0.01].to_csv("zzprogress.csv")

Keep exploring! Stay home, wash your hands, keep your distance.